# **Learn OpenUSD: Traversing Stages**

Welcome to the Jupyter notebook for *Learn OpenUSD: Traversing Stages*. This is where we will find all the Python activities related to this course. Before starting **Activity 1**, make sure to run the cell below.

>**NOTE**: Before starting make sure to run the cell below. This will install the relevant OpenUSD libraries that will be used through this notebook.

In [1]:
from utils.visualization import DisplayUSD
from utils.helperfunctions import create_new_stage

**Run the cell below to create the file that will be used for this module:**

In [2]:
from pxr import Usd, UsdGeom, UsdLux, UsdShade

stage: Usd.Stage = create_new_stage("assets/tons_of_prims.usda")

world: UsdGeom.Xform = UsdGeom.Xform.Define(stage, "/World")
stage.SetDefaultPrim(world.GetPrim())

box: UsdGeom.Xform = UsdGeom.Xform.Define(stage, world.GetPath().AppendPath("Box"))
geo_scope: UsdGeom.Scope = UsdGeom.Scope.Define(stage, box.GetPath().AppendPath("Geometry"))
box_geo: UsdGeom.Cube = UsdGeom.Cube.Define(stage, geo_scope.GetPath().AppendPath("Cube"))

mat_scope: UsdGeom.Scope = UsdGeom.Scope.Define(stage, box.GetPath().AppendPath("Materials"))
box_mat: UsdShade.Material = UsdShade.Material.Define(stage, mat_scope.GetPath().AppendPath("BoxMat"))

# Define a new Scope primitive at the path "/World/Environment" on the current stage
env: UsdGeom.Scope = UsdGeom.Scope.Define(stage, world.GetPath().AppendPath("Environment"))

# Define a new DistantLight primitive at the path "/World/Environment/SkyLight" on the current stage
distant_light: UsdLux.DistantLight = UsdLux.DistantLight.Define(stage, env.GetPath().AppendPath("SkyLight"))

stage.Save()
DisplayUSD("assets/tons_of_prims.usda", show_usd_code=True)

---

## **Activity 1**: Traversing Through the Stage

To traverse through the stage, we can use the [`Traverse()`](https://openusd.org/release/api/class_usd_stage.html#adba675b55f41cc1b305bed414fc4f178) method. This traversal will yield prims from the current stage in depth-first-traversal order.

**Add the following code to the cell below, then run the cell:**

```python
# Traverse through each prim (primitive) in the stage
for prim in stage.Traverse():
    # Print the path of each prim
    print(prim.GetPath())
```

In [3]:
# Import the Usd module from the pxr package
from pxr import Usd

# Open the USD stage from the specified file
stage: Usd.Stage = Usd.Stage.Open("assets/tons_of_prims.usda")

# Traverse through each prim (primitive) in the stage
for prim in stage.Traverse():
    # Print the path of each prim
    print(prim.GetPath())

/World
/World/Box
/World/Box/Geometry
/World/Box/Geometry/Cube
/World/Box/Materials
/World/Box/Materials/BoxMat
/World/Environment
/World/Environment/SkyLight


---

## **Activity 2**: Traversing USD Content for Specific Prim Types

Using [`Traverse()`](https://openusd.org/release/api/class_usd_stage.html#adba675b55f41cc1b305bed414fc4f178) can get costly as the stage scales.

We can filter based on the type of the prim. For example, we can check if the prim is of type `scope` or `xform`. To do this we pass the prim into the constructor method for the prim type we are interested in. For example,`UsdGeom.Scope(prim)` is equivalent to [`UsdGeom.Scope.Get(prim.GetStage(), prim.GetPath())`](https://openusd.org/release/api/class_usd_geom_scope.html#a538339c2aa462ebcf1eb07fed16f9be4) for a valid prim. If the prim's type does not match, it will return an invalid prim.

**Add the following code to the cell below, then run the cell:**

```python
# Traverse through each prim (primitive) in the stage
for prim in stage.Traverse():
    # Check if the prim is of type Scope
    print(UsdGeom.Scope(prim))
    if UsdGeom.Scope(prim):
        print("Scope Type: ", prim.GetName())
    # Check if the prim is of type Xform
    elif UsdGeom.Xform(prim):
        print("Xform Type: ", prim.GetName())
```

In [4]:
# Import necessary modules from the pxr package
from pxr import Usd, UsdGeom

# Open the USD stage from the specified file
stage: Usd.Stage = Usd.Stage.Open("assets/tons_of_prims.usda")


# Traverse through each prim (primitive) in the stage
for prim in stage.Traverse():
    # Check if the prim is of type Scope
    print(UsdGeom.Scope(prim))
    if UsdGeom.Scope(prim):
        print("Scope Type: ", prim.GetName())
    # Check if the prim is of type Xform
    elif UsdGeom.Xform(prim):
        print("Xform Type: ", prim.GetName())

UsdGeom.Scope(Usd.Prim(</World>))
Xform Type:  World
UsdGeom.Scope(Usd.Prim(</World/Box>))
Xform Type:  Box
UsdGeom.Scope(Usd.Prim(</World/Box/Geometry>))
Scope Type:  Geometry
UsdGeom.Scope(Usd.Prim(</World/Box/Geometry/Cube>))
UsdGeom.Scope(Usd.Prim(</World/Box/Materials>))
Scope Type:  Materials
UsdGeom.Scope(Usd.Prim(</World/Box/Materials/BoxMat>))
UsdGeom.Scope(Usd.Prim(</World/Environment>))
Scope Type:  Environment
UsdGeom.Scope(Usd.Prim(</World/Environment/SkyLight>))


---

## **Activity 3**: Traversing Through the Children of a Prim

Another way to be more efficient and targeted is to traverse through the children of a prim.

If you need to work within a specific scope or hierarchy in the stage, you can perform a traversal starting from a particular prim. Below shows a simple example on how to do this.

```python
from pxr import Usd

stage = Usd.Stage.Open('path/to/your.usda')
root_prim = stage.GetPrimAtPath("/root/path")

for prim in root_prim.GetChildren():
    # Perform operations on children of the root prim
    print(prim.GetPath())
```

Using [`Traverse()`](https://openusd.org/release/api/class_usd_stage.html#adba675b55f41cc1b305bed414fc4f178) can be a powerful tool, but for large stages, more efficient and targeted methods should be considered.

Let's take a look at how we can traverse through the children of the default prim.

**Copy the following code and paste it in the cell below:**

```python
# Iterate through all children of the default prim
for child in default_prim.GetAllChildren():
    # Print the path of each child prim
    print(child.GetPath())

# Get a specific child prim named "Environment" under the default prim
child_prim: Usd.Prim = default_prim.GetChild("Environment")
# Print the details of the child prim
print(child_prim)
```

In [5]:
# Import the `Usd` module from the `pxr` package:
from pxr import Usd

# Open the USD stage from the specified file:
stage: Usd.Stage = Usd.Stage.Open("assets/tons_of_prims.usda")

# Get the default prim (primitive) of the stage:
default_prim: Usd.Prim = stage.GetDefaultPrim()

# Iterate through all children of the default prim
for child in default_prim.GetAllChildren():
    # Print the path of each child prim
    print(child.GetPath())

# Get a specific child prim named "Environment" under the default prim
child_prim: Usd.Prim = default_prim.GetChild("Environment")
# Print the details of the child prim
print(child_prim)

/World/Box
/World/Environment
Usd.Prim(</World/Environment>)


---

## **Activity 4**: Traversing using `Usd.PrimRange`

[`Traverse()`](https://openusd.org/release/api/class_usd_stage.html#adba675b55f41cc1b305bed414fc4f178) will return a [`UsdPrimRange`](https://openusd.org/release/api/class_usd_prim_range.html) object. `UsdPrimRange` exposes pre- and post-order prim visitations allowing for a more involved traversals. It can also be used to perform actions such as pruning subtrees. [`Traverse()`](https://openusd.org/release/api/class_usd_stage.html#adba675b55f41cc1b305bed414fc4f178) is a convenience method that performs visitations on all prims in the composed scenegraph that are active, defined, loaded, and concrete. It is recommended to use [`UsdPrimRange`](https://openusd.org/release/api/class_usd_prim_range.html) over [`Traverse()`](https://openusd.org/release/api/class_usd_stage.html#adba675b55f41cc1b305bed414fc4f178) for traversals.

Let's see an example of [`UsdPrimRange`](https://openusd.org/release/api/class_usd_prim_range.html) in use.

**Add the following code to the cell below, then run the cell:**

``` python
prim_range = Usd.PrimRange(stage.GetDefaultPrim())
for prim in prim_range:
    print(prim.GetPath())
```

In [6]:
# Import the Usd module from the pxr package
from pxr import Usd

# Open the USD stage from the specified file
stage: Usd.Stage = Usd.Stage.Open("assets/tons_of_prims.usda")


prim_range = Usd.PrimRange(stage.GetDefaultPrim())
for prim in prim_range:
    print(prim.GetPath())

/World
/World/Box
/World/Box/Geometry
/World/Box/Geometry/Cube
/World/Box/Materials
/World/Box/Materials/BoxMat
/World/Environment
/World/Environment/SkyLight


[`Traverse()`](https://openusd.org/release/api/class_usd_stage.html#adba675b55f41cc1b305bed414fc4f178) will go through the entire stage. Unlike [`Traverse()`](https://openusd.org/release/api/class_usd_stage.html#adba675b55f41cc1b305bed414fc4f178), [`UsdPrimRange`](https://openusd.org/release/api/class_usd_prim_range.html) can also traverse through a pseudo root prim. To do this, we'd pass the prim in question as an argument to the function. The code below demonstrates this:

**Add the following code to the cell below, then run the cell:**

```python
prim_range = Usd.PrimRange(stage.GetPrimAtPath("/World/Environment"))
```

In [7]:
# Import the Usd module from the pxr package
from pxr import Usd

# Open the USD stage from the specified file
stage: Usd.Stage = Usd.Stage.Open("assets/tons_of_prims.usda")

prim_range = Usd.PrimRange(stage.GetPrimAtPath("/World/Environment"))

for prim in prim_range:
    print(prim.GetPath())

/World/Environment
/World/Environment/SkyLight


There are other ways to use [`UsdPrimRange`](https://openusd.org/release/api/class_usd_prim_range.html) such as passing in [`predicates`](https://openusd.org/release/api/prim_flags_8h.html#Usd_PrimFlags), you can find more information in the [Using `Usd.PrimRange` in Python](https://openusd.org/release/api/class_usd_prim_range.html#details) section of `UsdPrimRange`.

---

## **Activity 5**: Setting Prims as Active/Inactive

[Active/Inactive](https://openusd.org/release/glossary.html#active-inactive) prim is a behavior that is non-destructive and provides reversible prim deletion from a stage. By default, all prims are active. Active prims are visited by stage traversals.

If a prim is inactive that means it is not visited by stage traversals, and neither will its child prims.

To set whether the prim is active or inactive is by using [`SetActive()`](https://openusd.org/release/api/class_usd_prim.html#ac156eed30c42c013c4a4debf580ce17f). Giving the value of `False` will make it inactive and `True` to make it active.

**Add the following code to the cell below, then run the cell:**

```python
# Get the child prim named "Box" under the default prim and deactivate it
box = default_prim.GetChild("Box")
# Passing in False to SetActive() will set the prim as Inactive and passing in True will set the prim as active
box.SetActive(False)
```

In [8]:
from pxr import Usd

# Open the USD stage from the specified file
stage = Usd.Stage.Open("assets/tons_of_prims.usda")

# Get the default prim (primitive) of the stage
default_prim: Usd.Prim = stage.GetDefaultPrim()

# Get the child prim named "Box" under the default prim and deactivate it
box = default_prim.GetChild("Box")
# Passing in False to SetActive() will set the prim as Inactive and passing in True will set the prim as active
box.SetActive(False)

# Iterate through all children of the default prim
for child in default_prim.GetAllChildren():
    # Print the name of each child and their children (if any)
    print(child.GetName(), child.GetAllChildren())

# Get a specific child prim named "Environment" under the default prim
child_prim: Usd.Prim = default_prim.GetChild("Environment")
# Print the details of the child prim
print(child_prim)

Box []
Environment [Usd.Prim(</World/Environment/SkyLight>)]
Usd.Prim(</World/Environment>)
